In [3]:
import pandas as pd
import numpy as np
import statistics as stat
import random as random
import math as math

# Ear Phenotypes

In [4]:
useCols = [1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
df1 = pd.read_csv('../data/earphenotypesformatted.csv', na_values = {'a/n', 'a'}, usecols = useCols)
df2 = pd.read_csv('../data/earphenotypesformatted.csv', na_values = {'a/n', 'a'}, usecols = useCols)
fileList = [df1, df2]
traitList = ['Ear Width', 'Kernel Fill Length', 'Kernel Row Number', 'Kernels per Row', 'Ear Weight', 'Kernel Count', 'Cob Length', 'Cob Width', 'Cob Weight', '100 Kernel weight']

In [5]:
# What if we just concat and pivot
ears = pd.concat([df1, df2])
ears = ears.reset_index(drop=False)
ears = ears.replace({'22.94.':'22.94'})
ears['Kernel Mass'] = ears['Ear Weight'].astype(float) - ears['Cob Weight'].astype(float)
ears['QR Code'].astype(str)
ears['QR Code'] = ears['QR Code'].str.upper()
ears['earNum'] = ears.groupby('QR Code').cumcount()
ears = ears[ears['earNum'] < 4]
plots = pd.pivot(ears, index = 'QR Code', columns = 'earNum')
traitList.append('Kernel Mass')
plots = plots.astype(float)

In [ ]:
plots.dtypes

In [7]:
# Now that we have our nice data frame, we need to calculate the mean and standard error with all possible sets of 4 and all possible sets of 2 for each trait.
index30 = [1, 2, 3]
index31 = [0, 2, 3]
index32 = [0, 1, 2]
index20 = [0, 1]
index21 = [0, 2]
index22= [0, 3]
index23 = [1, 2]
index24= [1, 3]
index25= [2, 3]
index3List = [index30, index31, index32]
index3ListStr = ['30', '31', '32']
index2List = [index20, index21, index22, index23, index24, index25]
index2ListStr = ['20', '21', '22', '23', '24', '25']

results = pd.DataFrame()
for i in traitList:
    mean = i + 'mean' + '4'
    se = i + 'se' + '4'
    meanSeries = (plots[i][[0, 1, 2, 3]]).mean(axis = 1)
    results[mean] = meanSeries
    seSeries = (plots[i][[0, 1, 2, 3]]).sem(axis = 1)
    results[se] = seSeries
    
    for j in range(3):
        mean = i + 'mean' + index3ListStr[j]
        se = i + 'se' + index3ListStr[j]
        index = index3List[j]
        meanSeries = (plots[i][index]).mean(axis = 1)
        seSeries = (plots[i][index]).sem(axis = 1)
        results[mean] = meanSeries
        results[se] = seSeries
        
    for k in range(6):
        mean = i + 'mean' + index2ListStr[k]
        se = i + 'se' + index2ListStr[k]
        index = index2List[k]
        meanSeries = (plots[i][index]).mean(axis = 1)
        seSeries = (plots[i][index]).sem(axis = 1)
        results[mean] = meanSeries
        results[se] = seSeries

/var/folders/69/31h03yyn6kqcrntzcd0x_04c0000gn/T/ipykernel_6682/690879259.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  results[mean] = meanSeries
/var/folders/69/31h03yyn6kqcrntzcd0x_04c0000gn/T/ipykernel_6682/690879259.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  results[se] = seSeries
/var/folders/69/31h03yyn6kqcrntzcd0x_04c0000gn/T/ipykernel_6682/690879259.py:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Co

In [ ]:
meanSeries

In [ ]:
results

In [10]:
avgSE = pd.DataFrame()
for t in traitList:
    traitSE2 = t + '.SE.2'
    traitSE3 = t + '.SE.3'
    
    seCols2 = []
    for i in index2ListStr:
        colName = t + 'se' + i
        seCols2.append(colName)
     
    seCols3 = []
    for j in index3ListStr:
        colName = t + 'se' + j
        seCols3.append(colName)
        avgSE[traitSE2] = results[seCols2].mean(axis = 1)
        avgSE[traitSE3] = results[seCols3].mean(axis = 1)
    

In [ ]:
avgSE

In [11]:
corrPerSet = pd.DataFrame()
for t in traitList:
    fullMeanCol = t + 'mean' + '4'
    
    for i in index2ListStr:
        colName = t + 'mean' + i
        cor = results[colName].corr(results[fullMeanCol])
        corrPerSet[colName] = pd.Series()
        corrPerSet.at[0, colName] = cor
    for j in index3ListStr:
        colName = t + 'mean' + j
        cor = results[colName].corr(results[fullMeanCol])
        corrPerSet.at[0, colName] = cor
    

In [ ]:
corrPerSet.columns

In [12]:
summary = pd.DataFrame({'Set Size':[2, 3, 4]})

for t in traitList:
    sem = t + '.MeanSEM'
    corr = t + '.MeanCorr'
    sem2 = t + '.SE.2'
    sem3 = t + '.SE.3'
    semFull = t + 'se4'
    
    meanCols2 = []
    for i in index2ListStr:
        colName = t + 'mean' + i
        meanCols2.append(colName)
        
    meanCols3 = []
    for j in index3ListStr:
        colName = t + 'mean' + j
        meanCols3.append(colName)

    summary.at[0, sem] = avgSE[sem2].mean(axis = 0)
    summary.at[1, sem] = avgSE[sem3].mean(axis = 0)
    summary.at[2, sem] = results[semFull].mean(axis = 0)
    
    summary.at[0, corr] = corrPerSet[meanCols2].mean().values[0]
    summary.at[1, corr] = corrPerSet[meanCols3].mean().values[0]
    summary.at[2, corr] = 1

In [ ]:
# Export to csv
summary.to_csv('../analysis/RequiredNumberOfMeasurements.csv', index = False)

# NIR

In [13]:
nir = pd.read_excel('../data/NIRData.xlsx', sheet_name = 'in', dtype = {'Sample ID':str})
nir['protein'] = nir['Protein As is'] / (1 - (nir['Moisture'] / 100))
nir['oil'] = nir['Oil As is'] / ( 1 - (nir['Moisture'] / 100))
nir['fiber'] = nir['Fiber As is'] / (1 - (nir['Moisture'] / 100))
nir['ash'] = nir['Ash As is'] / (1 - (nir['Moisture'] / 100))
nir['starch'] = nir['Starch As is'] / (1 - (nir['Moisture'] / 100))
nir['qr'] = nir['Sample ID'].str.upper()
nir['qr'] = nir['qr'].astype(str)
nir = nir.rename(columns = {'Moisture':'moisture'})
traits = ['starch', 'protein', 'oil', 'fiber', 'ash', 'moisture']
nir = nir[['qr', 'starch', 'protein', 'oil', 'fiber', 'ash', 'moisture']]
nir = nir[~nir['qr'].str.contains('SYNGENTA')]

for t in traits:
    nir = nir[nir[t] >= 0]
nir['i'] = nir.groupby('qr').cumcount()
nir = nir[nir['i'] < 3]

nirWide = pd.pivot(nir, index = 'qr', columns = 'i')

In [16]:
index3 = [0, 1, 2]
index20 = [1, 2]
index21 = [0, 2]
index22 = [0, 1]
index2List = [index20, index21, index22]
index2ListStr = ['20', '21', '22']
nirResults = pd.DataFrame()

for t in traits:
    mean3 = t + '.Mean.3'
    se3 = t + '.SE.3'
    meanSeries = (nirWide[t][index3]).mean(axis = 1)
    seSeries = (nirWide[t][index3]).sem(axis = 1)
    nirResults[mean3] = meanSeries
    nirResults[se3] = seSeries
    
    for i in range(3):
        meanCol = t + '.Mean.' + index2ListStr[i]
        seCol = t + '.SE.' + index2ListStr[i]
        meanSeries = (nirWide[t][index2List[i]]).mean(axis = 1)
        seSeries = (nirWide[t][index2List[i]]).sem(axis = 1)
        nirResults[meanCol] = meanSeries
        nirResults[seCol] = seSeries   


,starch.Mean.3,starch.SE.3,starch.Mean.20,starch.SE.20,starch.Mean.21,starch.SE.21,starch.Mean.22,starch.SE.22,protein.Mean.3,protein.SE.3,...,ash.Mean.22,ash.SE.22,moisture.Mean.3,moisture.SE.3,moisture.Mean.20,moisture.SE.20,moisture.Mean.21,moisture.SE.21,moisture.Mean.22,moisture.SE.22
qr,,,,,,,,,,,,,,,,,,,,,
4101$ROW2$RANGE2$PHN46 X W606S,71.538868,NaN,NaN,NaN,71.538868,NaN,71.538868,NaN,9.011105,NaN,...,0.973030,NaN,5.450000,NaN,NaN,NaN,5.450,NaN,5.450,NaN
4102$ROW2$RANGE3$PHK56 X W606S,67.567393,0.055710,67.587251,0.090154,67.512386,0.015289,67.602541,0.074865,10.827169,0.199583,...,1.027232,0.004564,5.146667,0.072188,5.145,0.125,5.210,0.060,5.085,0.065
4103$ROW2$RANGE4$PHK56 X PHJ89,69.360047,0.342089,69.697273,0.099548,69.241207,0.555613,69.141660,0.456066,9.458068,0.121762,...,0.994419,0.004529,4.983333,0.042557,5.025,0.015,4.955,0.055,4.970,0.070
4104$ROW2$RANGE5$PHJ40 X LH82,68.701124,1.049531,69.248349,1.551188,69.203106,1.596431,67.651918,0.045243,10.347211,0.158729,...,1.148588,0.050151,5.096667,0.121974,4.985,0.085,5.195,0.125,5.110,0.210
4105$ROW2$RANGE6$PHP02 X PHK76,67.103147,0.174902,67.277716,0.018674,67.006526,0.252517,67.025199,0.271190,9.778549,0.377901,...,1.069750,0.072688,4.550000,0.111505,4.530,0.190,4.465,0.125,4.655,0.065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SCOTTSBLUFF$HYBRID-HIPS - MEDIUM NITROGEN$REP2$PLOT1336$ROW11$RANGE18$PHK76 X LH82,71.316724,0.216870,71.189699,0.304453,71.228009,0.342763,71.532462,0.038310,8.953693,0.145157,...,1.197312,0.005925,8.530000,0.025166,8.505,0.005,8.540,0.040,8.545,0.035
SCOTTSBLUFF$HYBRID-HIPS - MEDIUM NITROGEN$REP2$PLOT1337$ROW11$RANGE19$PHK76 X LH198,72.443342,0.046645,72.449750,0.080025,72.480150,0.049625,72.400125,0.030400,7.033401,0.035162,...,1.493527,0.006252,9.290000,0.030000,9.275,0.045,9.320,0.000,9.275,0.045
SCOTTSBLUFF$HYBRID-HIPS - MEDIUM NITROGEN$REP2$PLOT1338$ROW11$RANGE20$LH195 X LH123HT,70.177435,0.086074,70.130569,0.125047,70.138345,0.132823,70.263392,0.007776,7.483236,0.060356,...,1.339088,0.000148,9.576667,0.063596,9.540,0.090,9.550,0.100,9.640,0.010
